In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
def get_coords(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    coords = []
    for el in [x.text.strip() for x in soup.find_all('div', class_='col-lg-6 col-md-7 col-sm-7 col-xs-7')][:2]:
        neg = 1.0
        if el[-1] == 'W':
            neg = -1.0
        deg = el[:el.find('°')]
        mins = el[(el.find('°')+1):(el.find('\''))]
        secs = el[(el.find('\'')+1):el.find('.')]
        out = float(deg) + (float(mins)/60) + (float(secs)/3600)
        out = neg*out
        coords.append(out)
    return coords

In [3]:
def get_options(folder = ''):
    options = webdriver.ChromeOptions() 
    prefs = {'download.default_directory' : '/Users/arran/Documents/Spring_2023/arran_thesis/new_data/weather/' + folder}
    options.add_experimental_option('prefs', prefs)
    return options

In [4]:
all_coords = []

In [5]:
cdrive = '/opt/homebrew/Caskroom/chromedriver/109.0.5414.74/chromedriver'

### Code is repeated with slight variation for each weather station. Typically two seperate stations were needed, one that contained data on each month by day, and data by each month by year. Many files were needed for the former, as each year was a different file. To save time, and also reproducibility, this scrapping script was written. It goes to each individual web page (each page varies only by the year, simplifying the process), and then clicks the download button, storing the file in the correct location. It take ~30 minutes for all files to run.

### Additionally, it stores the coordinates of each station, which is needed when locating the nearest station to a fire.

In [6]:
#Kamloops, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2013-06-11%7C2022-10-18&dlyRange=2013-06-13%7C2022-10-17&mlyRange=%7C&StationID=51423&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&txtStationName=kamloops&timeframe=2&Day=18&Year=','&Month=10#']
for i in range(2013,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('kamloops/days_by_year'))
    driver.get(url[0] + str(i) + url[1])
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Kamloops, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1953-01-01%7C2013-06-13&dlyRange=1951-01-01%7C2013-06-12&mlyRange=1951-01-01%7C2013-06-01&StationID=1275&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=2&searchMethod=contains&Month=10&Day=19&txtStationName=kamloops&timeframe=3&Year=2013'
driver = webdriver.Chrome(cdrive, options=get_options('kamloops/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Kamloops'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3682783526.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('kamloops/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3682783526.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('kamloops/months_by_years'))


In [7]:
#Castlegar, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=1954-01-12%7C2013-05-16&dlyRange=1965-12-01%7C2022-10-18&mlyRange=1966-01-01%7C2007-02-01&StationID=1105&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=3&searchMethod=contains&Month=12&Day=19&txtStationName=castlegar&timeframe=2&Year=']
for i in range(1970,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('castlegar/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
all_coords.append(['Castlegar'] + get_coords(url[0]+'2022'))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3233308670.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('castlegar/days_by_year'))


In [8]:
#Nanaimo, days_by_year pt1
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=1954-01-01%7C2013-05-16&dlyRange=1947-03-01%7C2016-06-01&mlyRange=1947-01-01%7C2007-02-01&StationID=192&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=3&searchMethod=contains&Month=6&Day=19&txtStationName=nanaimo&timeframe=2&Year=']
for i in range(1980,2017):
    driver = webdriver.Chrome(cdrive, options=get_options('nanaimo/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Nanaimo, days_by_year pt2
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2013-05-16%7C2022-10-19&dlyRange=2016-06-02%7C2022-04-30&mlyRange=%7C&StationID=51318&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=2&searchMethod=contains&Month=4&Day=19&txtStationName=nanaimo&timeframe=2&Year=']
for i in range(2016,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('nanaimo/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
all_coords.append(['Nanaimo'] + get_coords(url[0]+'2022'))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1429337046.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('nanaimo/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1429337046.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('nanaimo/days_by_year'))


In [9]:
#Prince George, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2009-10-08%7C2022-10-19&dlyRange=2009-10-22%7C2022-10-18&mlyRange=%7C&StationID=48370&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=0&searchMethod=contains&Month=10&Day=19&txtStationName=prince+george&timeframe=2&Year=']
for i in range(2009,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('prince_george/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Prince George, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1953-01-01%7C2009-10-22&dlyRange=1942-07-01%7C2009-10-21&mlyRange=1942-01-01%7C2009-10-01&StationID=631&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=4&searchMethod=contains&Month=10&Day=19&txtStationName=prince+george&timeframe=3&Year=2009'
driver = webdriver.Chrome(cdrive, options=get_options('prince_george/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Prince George'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3063552557.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('prince_george/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3063552557.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('prince_george/months_by_years'))


In [10]:
#Smithers, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2010-02-02%7C2022-10-19&dlyRange=2010-04-08%7C2022-10-18&mlyRange=%7C&StationID=48628&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=0&searchMethod=contains&Month=10&Day=19&txtStationName=smithers&timeframe=2&Year=']
for i in range(2010,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('smithers/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Smithers, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1953-01-01%7C2012-04-12&dlyRange=1942-06-01%7C2018-11-03&mlyRange=1942-01-01%7C2010-04-01&StationID=487&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=3&searchMethod=contains&Month=10&Day=19&txtStationName=smithers&timeframe=3&Year=2010'
driver = webdriver.Chrome(cdrive, options=get_options('smithers/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Smithers'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/2940836551.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('smithers/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/2940836551.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('smithers/months_by_years'))


In [11]:
#Williams Lake, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2012-12-03%7C2022-10-19&dlyRange=2012-12-06%7C2022-10-18&mlyRange=%7C&StationID=50820&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=2&searchMethod=contains&Month=10&Day=19&txtStationName=williams+lake&timeframe=2&Year=']
for i in range(2012,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('williams_lake/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Williams Lake, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1961-01-01%7C2012-12-06&dlyRange=1960-12-01%7C2012-12-05&mlyRange=1961-01-01%7C2012-12-01&StationID=664&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=3&searchMethod=contains&Month=10&Day=19&txtStationName=williams+lake&timeframe=3&Year=2012'
driver = webdriver.Chrome(cdrive, options=get_options('williams_lake/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Williams Lake'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3837176062.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('williams_lake/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3837176062.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('williams_lake/months_by_years'))


In [12]:
#Tofino
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=1960-01-01%7C2014-05-01&dlyRange=1942-07-01%7C2022-11-02&mlyRange=1942-01-01%7C2007-02-01&StationID=277&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=2&searchMethod=contains&txtStationName=tofino&timeframe=2&Day=3&Year=']
for i in range(1970,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('tofino/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
all_coords.append(['Tofino'] + get_coords(url[0]+'2022'))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3216026093.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('tofino/days_by_year'))


In [13]:
#Port Hardy, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2013-06-11%7C2022-11-03&dlyRange=2013-04-13%7C2022-11-03&mlyRange=%7C&StationID=51319&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&Month=11&Day=3&txtStationName=port+hardy&timeframe=2&Year=']
for i in range(2013,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('port_hardy/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Port Hardy, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1953-01-01%7C2013-06-13&dlyRange=1944-02-01%7C2013-06-12&mlyRange=1944-01-01%7C2013-06-01&StationID=202&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=2&searchMethod=contains&Month=11&Day=3&txtStationName=port+hardy&timeframe=3&Year=2013'
driver = webdriver.Chrome(cdrive, options=get_options('port_hardy/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Port Hardy'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/2401970049.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('port_hardy/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/2401970049.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('port_hardy/months_by_years'))


In [14]:
#Vancouver, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2013-06-11%7C2022-11-03&dlyRange=2013-06-13%7C2022-11-03&mlyRange=%7C&StationID=51442&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=39&searchMethod=contains&Month=11&Day=3&txtStationName=vancouver&timeframe=2&Year=']
for i in range(2013,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('vancouver/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Vancouver, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1953-01-01%7C2013-06-13&dlyRange=1937-01-01%7C2013-06-12&mlyRange=1937-01-01%7C2013-06-01&StationID=889&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=37&searchMethod=contains&Month=11&Day=3&txtStationName=vancouver&timeframe=3&Year=2013'
driver = webdriver.Chrome(cdrive, options=get_options('vancouver/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Vancouver'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1277191850.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('vancouver/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1277191850.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('vancouver/months_by_years'))


In [15]:
#Whistler, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2005-01-17%7C2022-11-03&dlyRange=2005-06-13%7C2022-11-03&mlyRange=2005-11-01%7C2007-02-01&StationID=43443&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&Month=11&Day=3&txtStationName=whistler&timeframe=2&Year=']
for i in range(2007,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('whistler/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Whistler, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1983-01-01%7C2014-05-01&dlyRange=1976-11-01%7C2022-11-02&mlyRange=1976-01-01%7C2007-02-01&StationID=348&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=0&searchMethod=contains&Month=11&Day=3&txtStationName=whistler&timeframe=3&Year=2007'
driver = webdriver.Chrome(cdrive, options=get_options('whistler/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Whistler'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3139846427.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('whistler/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/3139846427.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('whistler/months_by_years'))


In [16]:
#Penticton, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2012-05-07%7C2022-11-03&dlyRange=2012-05-10%7C2022-11-03&mlyRange=%7C&StationID=50269&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&Month=11&Day=3&txtStationName=penticton&timeframe=2&Year=']
for i in range(2012,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('penticton/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Penticton, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1953-01-01%7C2012-05-10&dlyRange=1941-04-01%7C2012-05-10&mlyRange=1941-01-01%7C2012-05-01&StationID=1053&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=2&searchMethod=contains&Month=11&Day=3&txtStationName=penticton&timeframe=3&Year=2012'
driver = webdriver.Chrome(cdrive, options=get_options('penticton/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Penticton'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1686652516.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('penticton/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1686652516.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('penticton/months_by_years'))


In [17]:
#Prince Rupert, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=%7C&dlyRange=1959-08-01%7C2022-03-29&mlyRange=1959-01-01%7C2004-12-01&StationID=424&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=5&searchMethod=contains&Month=3&Day=3&txtStationName=prince+rupert&timeframe=2&Year=']
for i in range(1970,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('prince_rupert/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
all_coords.append(['Prince Rupert'] + get_coords(url[0]+'2022'))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/887249751.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('prince_rupert/days_by_year'))


In [18]:
#Dawson Creek, days_by_year pt1
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2009-08-27%7C2022-11-03&dlyRange=2009-09-03%7C2022-11-03&mlyRange=%7C&StationID=48208&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&Month=11&Day=3&txtStationName=dawson+creek&timeframe=2&Year=']
for i in range(2009,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('dawson_creek/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()

#Dawson Creek, days_by_year pt2
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=1964-01-23%7C2009-07-02&dlyRange=1968-03-01%7C2009-09-11&mlyRange=1968-01-01%7C2006-06-01&StationID=1404&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=2&searchMethod=contains&Month=9&Day=3&txtStationName=dawson+creek&timeframe=2&Year=']
for i in range(2007,2009):
    driver = webdriver.Chrome(cdrive, options=get_options('dawson_creek/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()

#Dawson Creek, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1964-01-23%7C2009-07-02&dlyRange=1968-03-01%7C2009-09-11&mlyRange=1968-01-01%7C2006-06-01&StationID=1404&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=2&searchMethod=contains&Month=11&Day=3&txtStationName=dawson+creek&timeframe=3&Year=2006'
driver = webdriver.Chrome(cdrive, options=get_options('dawson_creek/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Dawson Creek'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/2891005537.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('dawson_creek/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/2891005537.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('dawson_creek/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/2891005537.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('dawson_creek/months_by_years'))


In [19]:
#Nakusp, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=1994-02-01%7C2022-11-03&dlyRange=1991-01-01%7C2022-11-03&mlyRange=1991-02-01%7C2006-11-01&StationID=6840&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&Month=11&Day=3&txtStationName=nakusp&timeframe=2&Year=']
for i in range(1995,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('nakusp/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Nakusp, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=%7C&dlyRange=1912-03-01%7C1995-02-28&mlyRange=1912-01-01%7C1995-02-01&StationID=1131&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=0&searchMethod=contains&Month=11&Day=3&txtStationName=nakusp&timeframe=3&Year=1995'
driver = webdriver.Chrome(cdrive, options=get_options('nakusp/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Nakusp'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1678656225.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('nakusp/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1678656225.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('nakusp/months_by_years'))


In [20]:
#Dease Lake, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=1994-02-01%7C2022-11-03&dlyRange=1993-08-01%7C2022-11-03&mlyRange=1993-01-01%7C2007-02-01&StationID=6845&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&Month=11&Day=3&txtStationName=dease+lake&timeframe=2&Year=']
for i in range(2007,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('dease_lake/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Dease Lake, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1953-01-01%7C2012-11-15&dlyRange=1944-09-01%7C2011-12-15&mlyRange=1944-01-01%7C2007-02-01&StationID=1454&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=0&searchMethod=contains&Month=11&Day=3&txtStationName=dease+lake&timeframe=3&Year=2007'
driver = webdriver.Chrome(cdrive, options=get_options('dease_lake/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Dease Lake'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/2492838501.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('dease_lake/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/2492838501.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('dease_lake/months_by_years'))


In [21]:
#Bella Coola, days_by_year pt1
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2014-10-23%7C2022-11-03&dlyRange=2018-10-29%7C2022-11-03&mlyRange=%7C&StationID=52978&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=2&searchMethod=contains&Month=11&Day=3&txtStationName=bella+coola&timeframe=2&Year=']
for i in range(2018,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('bella_coola/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()

#Bella Coola, days_by_year pt2
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=1983-02-01%7C2013-08-15&dlyRange=1983-02-01%7C2017-11-18&mlyRange=1983-01-01%7C2007-02-01&StationID=381&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&Month=11&Day=3&txtStationName=bella+coola&timeframe=2&Year=']
for i in range(2007,2017):
    driver = webdriver.Chrome(cdrive, options=get_options('bella_coola/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()

#Bella Coola, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1983-02-01%7C2013-08-15&dlyRange=1983-02-01%7C2017-11-18&mlyRange=1983-01-01%7C2007-02-01&StationID=381&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&Month=11&Day=3&txtStationName=bella+coola&timeframe=3&Year=2007'
driver = webdriver.Chrome(cdrive, options=get_options('bella_coola/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Bella Coola'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1494319301.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('bella_coola/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1494319301.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('bella_coola/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1494319301.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('bella_coola/months_by_years'))


In [22]:
#Sparwood, days_by_year pt1
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2014-10-23%7C2022-11-03&dlyRange=2018-10-29%7C2022-11-03&mlyRange=%7C&StationID=52959&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=1&searchMethod=contains&Month=11&Day=3&txtStationName=sparwood&timeframe=2&Year=']
for i in range(2018,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('sparwood/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()

#Sparwood, days_by_year pt2
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=1980-03-03%7C2014-10-02&dlyRange=1980-03-01%7C2020-02-22&mlyRange=1980-01-01%7C2007-02-01&StationID=1207&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=0&searchMethod=contains&txtStationName=sparwood&timeframe=2&Day=3&Year=']
for i in range(2007,2018):
    driver = webdriver.Chrome(cdrive, options=get_options('sparwood/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()

#Sparwood, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1980-03-03%7C2014-10-02&dlyRange=1980-03-01%7C2020-02-22&mlyRange=1980-01-01%7C2007-02-01&StationID=1207&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2022&selRowPerPage=25&Line=0&searchMethod=contains&Month=11&Day=3&txtStationName=sparwood&timeframe=3&Year=2007'
driver = webdriver.Chrome(cdrive, options=get_options('sparwood/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Sparwood'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/512658330.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('sparwood/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/512658330.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('sparwood/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/512658330.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('sparwood/months_by_years'))


In [23]:
#Golden, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=1991-10-01%7C2014-08-07&dlyRange=1902-04-01%7C2023-01-11&mlyRange=1902-01-01%7C2007-02-01&StationID=1364&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2023&selRowPerPage=25&Line=2&searchMethod=contains&Month=12&Day=16&txtStationName=golden&timeframe=2&Year=']
for i in range(2007,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('golden/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Golden, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1991-10-01%7C2014-08-07&dlyRange=1902-04-01%7C2023-01-11&mlyRange=1902-01-01%7C2007-02-01&StationID=1364&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2023&selRowPerPage=25&Line=2&searchMethod=contains&txtStationName=golden&timeframe=3&Month=1&Day=16&Year=2006#'
driver = webdriver.Chrome(cdrive, options=get_options('golden/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Golden'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1639933978.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('golden/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1639933978.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('golden/months_by_years'))


In [24]:
#Fort Nelson, days_by_year
url = ['https://climate.weather.gc.ca/climate_data/daily_data_e.html?hlyRange=2012-11-08%7C2023-01-16&dlyRange=2012-11-08%7C2023-01-16&mlyRange=%7C&StationID=50819&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2023&selRowPerPage=25&Line=1&searchMethod=contains&txtStationName=fort+nelson&timeframe=2&Day=16&Year=']
for i in range(2012,2022):
    driver = webdriver.Chrome(cdrive, options=get_options('fort_nelson/days_by_year'))
    driver.get(url[0] + str(i))
    driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
    time.sleep(1)
    driver.close()
    
#Fort Nelson, months_by_years
url = 'https://climate.weather.gc.ca/climate_data/monthly_data_e.html?hlyRange=1953-01-01%7C2012-11-08&dlyRange=1937-09-01%7C2012-11-14&mlyRange=1937-01-01%7C2012-11-01&StationID=1455&Prov=BC&urlExtension=_e.html&searchType=stnName&optLimit=yearRange&StartYear=1840&EndYear=2023&selRowPerPage=25&Line=2&searchMethod=contains&txtStationName=fort+nelson&timeframe=3&Month=1&Day=16&Year=2011#'
driver = webdriver.Chrome(cdrive, options=get_options('fort_nelson/months_by_years'))
driver.get(url)
driver.find_element(By.XPATH, '//*[@id="climateNav"]/div[3]/section/div[1]/form/input[7]').click()
time.sleep(1)
driver.close()
all_coords.append(['Fort Nelson'] + get_coords(url))

/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1800713939.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('fort_nelson/days_by_year'))
/var/folders/f9/9vn_4nrs38q_5092cm0ssfhm0000gn/T/ipykernel_79954/1800713939.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(cdrive, options=get_options('fort_nelson/months_by_years'))


In [25]:
out_coords = pd.DataFrame(all_coords)
out_coords.to_csv('weather_stations_coordinates.csv', index=False)